In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd

titanic_df = pd.read_csv("../../data/titanic_train.csv")
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:

# 전처리
mean_data = titanic_df["Age"].mean()
titanic_df["Age"] = titanic_df["Age"].fillna(titanic_df["Age"].mean())
titanic_df["Cabin"] = titanic_df["Cabin"].fillna("N")
titanic_df["Embarked"] = titanic_df["Embarked"].fillna("N")

from sklearn.preprocessing import LabelEncoder

le_Sex = LabelEncoder()
le_Sex.fit(titanic_df["Sex"])
tmp = le_Sex.transform(titanic_df["Sex"])
titanic_df["Sex"] = tmp

le_Embarked = LabelEncoder()
le_Embarked.fit(titanic_df["Embarked"])
tmp = le_Embarked.transform(titanic_df["Embarked"])
titanic_df["Embarked"] = tmp

drop_feature = ["PassengerId", "Name", "Ticket", "Cabin"]
titanic_df = titanic_df.drop(drop_feature, axis=1)

y = titanic_df["Survived"]
X = titanic_df.drop(["Survived"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [16]:
# 모델 선택
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

xgb_model = XGBClassifier()

xgb_params = {
    # "n_estimators": [100, 200, 300],  # 부스팅 단계 수
    # "n_estimators": list(range(100, 301)),  # 부스팅 단계 수
    "n_estimators": np.arange(100, 501, 50),  # 부스팅 단계 수
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],  # 목표를 향해 가는 보폭의 크기
    "max_depth": np.arange(3, 10),
    "min_child_weight": np.arange(1, 11),
    "subsample": np.arange(0.6, 1.01, 0.1),  # 샘플링 비율
    "colsample_bytree": np.arange(0.6, 1.01, 0.1),  # 특정 트리의 샘플 비율
}

xgb_search = RandomizedSearchCV(
    estimator=xgb_model,  # 학습할 모델
    param_distributions=xgb_params,  # 하이퍼 파라미터 설정
    n_iter=50,  # 랜덤 탐색 횟수
    # n_iter=500,  # 랜덤 탐색 횟수
    scoring="accuracy",
    cv=5,
    verbose=2, # 진행상황 출력
    random_state=11,
    n_jobs=-1  # 최대 cpu 사용
)
xgb_params

{'n_estimators': array([100, 150, 200, 250, 300, 350, 400, 450, 500]),
 'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
 'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
 'min_child_weight': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ]),
 'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ])}

In [17]:
xgb_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2,
                                                          0.3],
                                        'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                                        'min_child_weight': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([100, 150, 200, 250, 300, 350, 400, 450, 500]),
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=11, scoring='accuracy', verbose=2)

In [18]:
# 베스트 모델에 대한 파라미터

best_params = xgb_search.best_params_
print(f"best_params: {best_params}")

best_score = xgb_search.best_score_
print(f"best_score: {best_score}")

best_model = xgb_search.best_estimator_
print(f"best_model: {best_model}")

best_params: {'subsample': np.float64(0.7), 'n_estimators': np.int64(450), 'min_child_weight': np.int64(4), 'max_depth': np.int64(9), 'learning_rate': 0.05, 'colsample_bytree': np.float64(0.7999999999999999)}
best_score: 0.8300600807643062
best_model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=np.float64(0.7999999999999999), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=np.int64(9), max_leaves=None,
              min_child_weight=np.int64(4), missing=nan,
              monotone_constraints=None, multi_strategy=None,
              n_estimators=np.

In [22]:
import numpy as np

print(le_Sex.transform(["male", "female"]))
# 샘플 직접 입력하기
dicaprio = np.array([[1, 1, 24, 0, 0, 8, 3]])
best_model.predict(dicaprio)

[1 0]


array([0])